In [1]:
using GynC
using JLD
using KernelDensity
using Plots; gr()

Plots.GRBackend()

In [2]:
using Distributions
import Distributions: pdf, rand

type MeasErr <: Distribution
    rhos
end

function pdf(m::MeasErr, delta::Matrix)
  d = 0.
  nonnans = 0
  for i=1:4
    for j=1:31
      v = delta[j,i]
      isnan(v) && continue
      nonnans += 1
      d += pdf(m.rhos[i], v)
    end
  end
  d/nonnans
end

function rand(m::MeasErr)
  [rand(m.rhos[j]) for i=1:31, j=1:4]
end

err = MeasErr([Normal(std) for std in GynC.model_measerrors]);

In [3]:
@time s=JLD.load("../data/0911/allsamples.jld")["samples"];

 18.394159 seconds (2.67 M allocations: 7.886 GB, 37.18% gc time)


In [4]:
function subsample(samplings::Vector{Matrix{Float64}}, n::Int, burnin::Int)
    res = Vector{Vector{Float64}}()
    nsamplings = length(samplings)
    for sampling in samplings
        nsamples = size(sampling, 1)
        step = floor(Int,(nsamples - burnin) / n * nsamplings)
        for i = burnin+1:step:nsamples
            s = sampling[i,:]
            push!(res, sampling[i,:])
        end
    end
    res
end

@time xs = subsample(s, 1000, 100_000);
@show length(xs);

  0.070859 seconds (32.72 k allocations: 2.442 MB)
length(xs) = 1035


In [71]:
datas = vcat([GynC.Lausanne(i).data for i=1:20]);

In [43]:
@time ys = map(x->GynC.forwardsol(x)[:,GynC.measuredinds], xs);

naninds = find(x->any(isnan(x)), ys)
nonnaninds = deleteat!(collect(1:length(ys)), naninds)

xs = xs[nonnaninds]
ys = ys[nonnaninds]

@time zs = map(y->y+rand(err), ys);

  5.949662 seconds (26.70 M allocations: 551.557 MB, 5.14% gc time)
  0.082654 seconds (286.69 k allocations: 9.201 MB)


In [73]:
m = GynC.LikelihoodModel(xs, ys, zs, datas, err);

In [74]:
w0 = ones(length(ys)) / length(xs);

In [106]:
niters = 100
reg = 1 - 1/length(datas)
reg=1
h = 1
@time ws = GynC.mple(m, w0, niters, reg, h)
w = ws[end];

  3.142641 seconds (587.68 k allocations: 18.192 MB, 0.64% gc time)


In [107]:
species = 8
plot()
for w in ws
  k=kde(map(x->x[species], xs), weights=w)
  plot!(k.x, k.density)
end
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.4296,381.012 43.6995,381.012 43.9695,381.012 44.2395,381.012 44.5095,381.012 44.7794,381.012 45.0494,381.012 45.3194,381.012 45.5894,381.012 45.8593,381.012 
 46.1293,381.012 46.3993,381.012 46.6692,381.012 46.9392,381.012 47.2092,381.011 47.4792,381.011 47.7491,381.011 48.0191,381.011 48.2891,381.011 48.559,381.011 
 48.829,381.01 49.099,381.01 49.369,381.01 49.6389,381.01 49.9089,381.01 50.1789,381.009 50.4489,381.009 50.7188,381.009 50.9888,381.008 51.2588,381.008 
 51.5287,381.008 51.7987,381.007 52.0687,381.007 52.3387,381.007 52.6086,381.006 52.8786,381.006 53.1486,381.005 53.4185,381.005 53.6885,381.004 53.9585,381.004 
 54.2285,381.003 54.4984,381.002 54.7684,381.002 55.0384,381.001 55.3084,381.001 55.5783,381 55.8483,380.999 56.1183,380.998 56.3882,380.997 56.6582,380.997 
 56.9282,380.996 57.1982,380.995 57.4681,380.994 57.7381,380.993 58.0081,380.991 58.2781,380.99 58.548,380.989 58.818,380.988 59.088,380.987 59.3579,380.985 
 59.6279,380.984 59.8979,380.982 60.1679,380.981 60.4378,380.979 60.7078,380.977 60.9778,380.975 61.2477,380.974 61.5177,380.972 61.7877,380.97 62.0577,380.967 
 62.3276,380.965 62.5976,380.963 62.8676,380.96 63.1376,380.958 63.4075,380.955 63.6775,380.953 63.9475,380.95 64.2174,380.947 64.4874,380.944 64.7574,380.94 
 65.0274,380.937 65.2973,380.933 65.5673,380.93 65.8373,380.926 66.1072,380.922 66.3772,380.918 66.6472,380.913 66.9172,380.909 67.1871,380.904 67.4571,380.899 
 67.7271,380.894 67.9971,380.889 68.267,380.884 68.537,380.878 68.807,380.872 69.0769,380.866 69.3469,380.859 69.6169,380.853 69.8869,380.846 70.1568,380.839 
 70.4268,380.831 70.6968,380.823 70.9668,380.815 71.2367,380.807 71.5067,380.798 71.7767,380.789 72.0466,380.78 72.3166,380.77 72.5866,380.76 72.8566,380.749 
 73.1265,380.738 73.3965,380.727 73.6665,380.715 73.9364,380.703 74.2064,380.691 74.4764,380.678 74.7464,380.664 75.0163,380.65 75.2863,380.636 75.5563,380.621 
 75.8263,380.605 76.0962,380.589 76.3662,380.572 76.6362,380.555 76.9061,380.537 77.1761,380.518 77.4461,380.499 77.7161,380.479 77.986,380.459 78.256,380.438 
 78.526,380.416 78.796,380.393 79.0659,380.369 79.3359,380.345 79.6059,380.32 79.8758,380.294 80.1458,380.267 80.4158,380.239 80.6858,380.21 80.9557,380.181 
 81.2257,380.15 81.4957,380.119 81.7656,380.086 82.0356,380.052 82.3056,380.017 82.5756,379.981 82.8455,379.944 83.1155,379.906 83.3855,379.867 83.6555,379.826 
 83.9254,379.784 84.1954,379.74 84.4654,379.696 84.7353,379.65 85.0053,379.602 85.2753,379.553 85.5453,379.503 85.8152,379.451 86.0852,379.397 86.3552,379.342 
 86.6251,379.285 86.8951,379.227 87.1651,379.166 87.4351,379.104 87.705,379.04 87.975,378.975 88.245,378.907 88.515,378.837 88.7849,378.765 89.0549,378.692 
 89.3249,378.616 89.5948,378.538 89.8648,378.458 90.1348,378.375 90.4048,378.29 90.6747,378.203 90.9447,378.113 91.2147,378.021 91.4847,377.927 91.7546,377.83 
 92.0246,377.73 92.2946,377.627 92.5645,377.522 92.8345,377.414 93.1045,377.303 93.3745,377.189 93.6444,377.072 93.9144,376.951 94.1844,376.828 94.4543,376.702 
 94.7243,376.572 94.9943,376.439 95.2643,376.303 95.5342,376.163 95.8042,376.019 96.0742,375.872 96.3442,375.721 96.6141,375.567 96.8841,375.409 97.1541,375.246 
 97.424,375.08 97.694,374.91 97.964,374.736 98.234,374.557 98.5039,374.374 98.7739,374.187 99.0439,373.995 99.3138,373.799 99.5838,373.598 99.8538,373.393 
 100.124,373.182 100.394,372.967 100.664,372.747 100.934,372.522 101.204,372.292 101.474,372.057 101.744,371.816 102.014,371.57 102.284,371.318 102.554,371.061 
 102.823,370.799 103.093,370.531 103.363,370.256 103.633,369.976 103.903,369.69 104.173,369.398 104.443,369.1 104.713,368.796 104.983,368.485 105.253,368.167 
 105.523,367.844 105.793,367.513 106.063,

In [108]:
plot(map(w->GynC.hz(m,w), ws)) |> display
plot(map(w->GynC.logl(m,w), ws))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 1.9 
 
 
 2.0 
 
 
 2.1 
 
 
 2.2 
 
 
 2.3 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 34.5407,3.93701 40.2126,58.7854 45.8846,109.248 51.5565,150.279 57.2285,181.828 62.9004,206.056 68.5723,225.004 74.2443,239.779 79.9162,251.431 85.5882,260.812 
 91.2601,268.617 96.9321,275.279 102.604,281.027 108.276,286.103 113.948,290.536 119.62,294.481 125.292,298.037 130.964,301.25 136.636,304.169 142.308,306.846 
 147.98,309.312 153.651,311.61 159.323,313.772 164.995,315.839 170.667,317.811 176.339,319.683 182.011,321.468 187.683,323.186 193.355,324.826 199.027,326.403 
 204.699,327.94 210.371,329.436 216.043,330.886 221.715,332.29 227.387,333.65 233.059,334.969 238.731,336.231 244.403,337.437 250.074,338.592 255.746,339.696 
 261.418,340.77 267.09,341.822 272.762,342.855 278.434,343.865 284.106,344.845 289.778,345.787 295.45,346.709 301.122,347.616 306.794,348.51 312.466,349.387 
 318.138,350.256 323.81,351.117 329.482,351.97 335.154,352.808 340.826,353.631 346.498,354.438 352.169,355.235 357.841,356.024 363.513,356.808 369.185,357.582 
 374.857,358.337 380.529,359.081 386.201,359.818 391.873,360.546 397.545,361.257 403.217,361.949 408.889,362.624 414.561,363.284 420.233,363.931 425.905,364.569 
 431.577,365.201 437.249,365.825 442.921,366.448 448.592,367.067 454.264,367.683 459.936,368.294 465.608,368.898 471.28,369.495 476.952,370.088 482.624,370.67 
 488.296,371.243 493.968,371.802 499.64,372.352 505.312,372.897 510.984,373.439 516.656,373.98 522.328,374.52 528,375.051 533.672,375.574 539.344,376.093 
 545.016,376.607 550.687,377.116 556.359,377.622 562.031,378.121 567.703,378.617 573.375,379.108 579.047,379.596 584.719,380.074 590.391,380.546 596.063,381.015 
 
 "/>
 
 
 
 
 y1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 -53 
 
 
 -52 
 
 
 -51 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 34.5407,381.015 40.2126,321.786 45.8846,269.925 51.5565,226.866 57.2285,192.085 62.9004,163.899 68.5723,140.951 74.2443,122.98 79.9162,108.65 85.5882,96.94 
 91.2601,87.1194 96.9321,78.6374 102.604,71.2179 108.276,64.5992 113.948,58.897 119.62,53.9042 125.292,49.539 130.964,45.7489 136.636,42.4664 142.308,39.6653 
 147.98,37.3471 153.651,35.3225 159.323,33.4475 164.995,31.6862 170.667,30.0205 176.339,28.5208 182.011,27.177 187.683,25.9494 193.355,24.9079 199.027,23.987 
 204.699,23.1161 210.371,22.2752 216.043,21.5176 221.715,20.8338 227.387,20.1972 233.059,19.6068 238.731,19.0768 244.403,18.5898 250.074,18.1289 255.746,17.6802 
 261.418,17.2312 267.09,16.7835 272.762,16.346 278.434,15.9117 284.106,15.5107 289.778,15.1999 295.45,14.9488 301.122,14.7209 306.794,14.5169 312.466,14.3326 
 318.138,14.1569 323.81,13.991 329.482,13.8323 335.154,13.6779 340.826,13.5237 346.498,13.3652 352.169,13.1949 357.841,13.0061 363.513,12.8008 369.185,12.6136 
 374.857,12.4305 380.529,12.2282 386.201,12.0103 391.873,11.7902 397.545,11.561 403.217,11.3298 408.889,11.0782 414.561,10.8003 420.233,10.5013 425.905,10.1955 
 431.577,9.90718 437.249,9.6184 442.921,9.32339 448.592,9.00625 454.264,8.69119 459.936,8.36558 465.608,8.03674 471.28,7.72777 476.952,7.42481 482.624,7.15444 
 488.296,6.89808 493.968,6.64958 499.64,6.40855 505.312,6.18652 510.984,5.9808 516.656,5.78669 522.328,5.60472 528,5.43515 533.672,5.28713 539.344,5.15226 
 545.016,5.02966 550.687,4.92016 556.359,4.81331 562.031,4.7028 567.703,4.58455 573.375,4.45899 579.047,4.32508 584.719,4.19464 590.391,4.0652 596.063,3.93701 
 
 "/>
 
 
 
 
 y1